In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# 物性値
NU = 0.3               # ポアソン比
YOUNG_MODULE = 210000  # ヤング率
t = 1                  # 板厚

# modelのインポート
model = pd.read_csv("model_large.csv")
node_num = max(model["tri_node_1"].max(), model["tri_node_2"].max(), model["tri_node_3"].max())
dof_node = 2
dof_total = node_num * dof_node
node_tria = 3
dof_tria3 = node_tria * dof_node

# 平面歪みのDマトリクス
d_coef = YOUNG_MODULE / ((1 - 2 * NU) * (1 + NU))

d_mat = d_coef * np.array([
    [1-NU, NU, 0],
    [NU, 1-NU, 0],
    [0, 0, (1-2*NU)/2]
])


In [ ]:
# Bマトリクス作成

b_mat = [None]
tri_arias = [None]

for element_no in model["element_no"].tolist():
    
    x_1 = int(model[model["element_no"] == element_no]["tri_node_1_x"])
    y_1 = int(model[model["element_no"] == element_no]["tri_node_1_y"])
    x_2 = int(model[model["element_no"] == element_no]["tri_node_2_x"])
    y_2 = int(model[model["element_no"] == element_no]["tri_node_2_y"])
    x_3 = int(model[model["element_no"] == element_no]["tri_node_3_x"])
    y_3 = int(model[model["element_no"] == element_no]["tri_node_3_y"])

    tri_aria = (x_1 * y_2 - x_1 * y_3 + x_2 * y_3 - x_2 * y_1 + x_3 * y_1 - x_3 * y_2) / 2
    coef = 1 / (2 * tri_aria)
    
    b_mat_one = np.array([
        [coef * (y_2 - y_3), 0, coef * (y_3 - y_1), 0, coef * (y_1 - y_2), 0],
        [0, coef * (x_3 - x_2), 0, coef * (x_1 - x_3), 0, coef * (x_2 - x_1)],
        [coef * (x_3 - x_2), coef * (y_2 - y_3), coef * (x_1 - x_3), coef * (y_3 - y_1), coef * (x_2 - x_1), coef * (y_1 - y_2)]
    ])
    
    b_mat.append(b_mat_one)
    tri_arias.append(tri_aria)


In [ ]:
# 剛性マトリクスKeの作成(要素ごと)

ke_mat = [None]

for element_no in model["element_no"].tolist():
    
    ke_one = t * tri_arias[element_no] * b_mat[element_no].T @ d_mat.T @ b_mat[element_no]
    ke_mat.append(ke_one)
    

In [ ]:
# 全体剛性マトリクスKの作成

k_mat = np.zeros((dof_total, dof_total))  # Kマトリクス, 0で初期化

for element_no in model["element_no"].tolist():
# for element_no in range(1):
#     element_no = 1
    
    for r in range(dof_tria3):
        # 要素内接点順(1, 2, 3)を求める
        er = math.ceil((r + 1) / dof_node)
        
        # 要素番号と要素内接点順からトータル接点番号を求める
        nr = 0
        if er == 1:
            nr = int(model[model["element_no"] == element_no]["tri_node_1"])
        elif er == 2:
            nr = int(model[model["element_no"] == element_no]["tri_node_2"])
        elif er == 3:
            nr = int(model[model["element_no"] == element_no]["tri_node_3"])

        # x成分かy成分か判断
        mr = (r + 1) % dof_node  # x(偶数)なら1, y(奇数)なら0
        
        # 全体合成マトリクスの番号を求める
        rt = nr * dof_node - mr - 1  # rt: Kマトリクスの行番号(np.array合わせで0から)
        
        for c in range(dof_tria3):
            # 要素内接点順(1, 2, 3)を求める
            ec = math.ceil((c + 1) / dof_node)
            
            # 要素番号と要素内接点順からトータル接点番号を求める
            nc = 0
            if ec == 1:
                nc = int(model[model["element_no"] == element_no]["tri_node_1"])
            elif ec == 2:
                nc = int(model[model["element_no"] == element_no]["tri_node_2"])
            elif ec == 3:
                nc = int(model[model["element_no"] == element_no]["tri_node_3"])

            # x成分かy成分か判断
            mc = (c + 1) % dof_node  # x(偶数)なら1, y(奇数)なら0
        
            # 全体合成マトリクスの番号を求める        
            ct = nc * dof_node - mc - 1  # ct: Kマトリクスの列番号(np.array合わせで0から)

            # KマトリクスにKeマトリクス足しこむ(加算する)
            k_mat[ct, rt] += ke_mat[element_no][c, r]


In [ ]:
# 境界条件(Boundary condition)の設定

f_vec = np.zeros(dof_total)  # 全体荷重ベクトル
u_vec = np.zeros(dof_total)  # 全体変位ベクトル
um_vec = []                  # 拘束目印ベクトル(拘束部分をTrueにする)

# ここで境界条件の設定を行う=======================
f_vec[29] = -100

for i in range(15):
    if i == 0:
        um_vec.append(True)
        um_vec.append(True)
    elif i == 9:
        um_vec.append(True)
        um_vec.append(True)
    elif i == 10:
        um_vec.append(True)
        um_vec.append(True)
    else:
        um_vec.append(False)
        um_vec.append(False)


# =============================================


# Kマトリクスのコピーを作成
kc_mat = k_mat.copy()

for r in range(node_num * dof_node):  # 行方向に順に処理

    if um_vec[r]:  # 拘束条件が設定されている成分に対して処理
        print(f"{r}行ベクトル拘束条件設定")
        
        for rr in range(dof_total):  # 変位拘束成分以外の荷重ベクトルを修正
            if rr != r:
                f_vec[rr] = f_vec[rr] - kc_mat[r, rr] * u_vec[r]
    
        # 拘束条件の存在する行の成分を0にする
        for rr in range(dof_total):
            kc_mat[r, rr] = 0.0
        
        # 拘束条件の存在する列の成分を0にする
        for cc in range(dof_total):
            kc_mat[cc, r] = 0.0

        # 対角成分を1にする
        kc_mat[r, r] = 1
    
        # 変位拘束成分に対応する荷重ベクトルを修正
        f_vec[r] = u_vec[r]


In [ ]:
#連立方程式を解く(ガウス法)

u_vec_ans = np.linalg.solve(kc_mat, f_vec)

In [ ]:
# 節点反力を計算

f_ref_vec = np.zeros(dof_total)

for r in range(dof_total):
    for c in range(dof_total):
        f_ref_vec[r] += k_mat[c, r] * u_vec_ans[c]


        
# ひずみの計算
# for element_no in model["element_no"].tolist():
#     for n in node_tria3:
        



In [ ]:
x_before = [0, 1, 2, 3, 4, 4, 4, 3, 2, 1, 0, 0, 0]
y_before = [0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 1, 0]

off = 0
x_u = [
    u_vec_ans[0 + off],
    u_vec_ans[2 + off],
    u_vec_ans[4 + off],
    u_vec_ans[6 + off],
    u_vec_ans[8 + off],
    u_vec_ans[10 + off],
    u_vec_ans[28 + off],
    u_vec_ans[26 + off],
    u_vec_ans[24 + off],
    u_vec_ans[22 + off],
    u_vec_ans[20 + off],
    u_vec_ans[18 + off],
    0
]

off = 1
y_u = [
    u_vec_ans[0 + off],
    u_vec_ans[2 + off],
    u_vec_ans[4 + off],
    u_vec_ans[6 + off],
    u_vec_ans[8 + off],
    u_vec_ans[10 + off],
    u_vec_ans[28 + off],
    u_vec_ans[26 + off],
    u_vec_ans[24 + off],
    u_vec_ans[22 + off],
    u_vec_ans[20 + off],
    u_vec_ans[18 + off],
    0
]

x_after = x_before.copy()
y_after = y_before.copy()

rate = 100

for i, d in enumerate(x_u):
    x_after[i] += d * rate
    
for i, d in enumerate(y_u):
    y_after[i] += d * rate

In [ ]:
plt.plot(x_before, y_before, marker='.')
plt.plot(x_after, y_after, marker='.')
plt.show()